# Day18_1: BERT 파인튜닝 - 정답

---

In [ ]:
# 필수 라이브러리 임포트
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

---

## Q1. BERT 개념 이해하기

**문제**: BERT가 "Bidirectional"이라고 불리는 이유를 설명하고, 기존 단방향 모델(GPT 등)과의 차이점을 서술하세요.

In [ ]:
# 정답
answer_q1 = """
BERT가 'Bidirectional'이라고 불리는 이유:

1. 양방향 문맥 이해:
   - BERT는 Transformer의 Encoder만 사용하여 입력 시퀀스의 모든 토큰을 동시에 처리합니다.
   - 각 토큰은 Self-Attention을 통해 좌측과 우측 모든 토큰의 정보를 참조할 수 있습니다.
   - 따라서 "진정한 양방향(truly bidirectional)" 모델이라고 합니다.

2. 기존 단방향 모델(GPT)과의 차이:
   
   | 특성 | GPT (단방향) | BERT (양방향) |
   |------|-------------|---------------|
   | 문맥 참조 | 좌측 토큰만 | 좌측 + 우측 모두 |
   | Masking | Causal Masking | 없음 (MLM용 마스킹만) |
   | 학습 방식 | 다음 토큰 예측 | 마스킹된 토큰 예측 |
   | 적합한 태스크 | 텍스트 생성 | 분류, 이해 |

3. 예시:
   문장: "나는 [MASK]에 가서 돈을 찾았다."
   
   - GPT: '나는'만 보고 [MASK] 예측 -> 어려움
   - BERT: '나는' + '돈을 찾았다' 모두 보고 예측 -> '은행' 예측 가능
"""

print(answer_q1)

In [ ]:
# 검증: 양방향 문맥의 중요성 시연
example = "나는 [MASK]에 가서 돈을 찾았다."

print("양방향 문맥 이해 예시:")
print(f"문장: {example}")
print("\n단방향(GPT 스타일):")
print("  - '나는'만으로는 [MASK]가 무엇인지 알기 어려움")
print("\n양방향(BERT):")
print("  - '돈을 찾았다'를 함께 보면 '은행'임을 알 수 있음")

### 풀이 설명

- **핵심 개념**: BERT는 Transformer Encoder의 Self-Attention을 사용하여 모든 토큰이 서로를 참조할 수 있습니다.
- **대안**: ELMo는 양방향 LSTM을 사용하지만, 좌->우와 우->좌를 별도로 학습한 후 결합합니다. BERT는 동시에 양방향을 처리합니다.
- **실무 팁**: 분류, NER, QA 등 "이해" 태스크에는 BERT가 적합하고, 텍스트 생성에는 GPT가 적합합니다.

---

## Q2. MLM과 NSP 이해하기

**문제**: BERT의 두 가지 사전학습 과제인 MLM과 NSP의 역할을 각각 한 문장으로 설명하세요.

In [ ]:
# 정답
answer_q2 = """
MLM (Masked Language Model):
- 입력 토큰의 15%를 마스킹하고 원래 토큰을 예측하게 하여,
  BERT가 양방향 문맥을 바탕으로 단어의 의미와 문법을 학습하게 합니다.

NSP (Next Sentence Prediction):
- 두 문장이 연속된 문장인지 예측하게 하여,
  BERT가 문장 간의 관계와 담화의 일관성을 이해하게 합니다.

역할 요약:
- MLM: 단어 수준의 언어 이해 (문맥적 의미, 문법)
- NSP: 문장 수준의 언어 이해 (문장 관계, 논리적 흐름)
"""

print(answer_q2)

In [ ]:
# 검증: MLM과 NSP 예시
print("MLM 예시:")
print("  원본: '오늘 날씨가 좋다'")
print("  입력: '오늘 [MASK]가 좋다'")
print("  예측: '날씨' (양방향 문맥 활용)")

print("\nNSP 예시:")
print("  문장 A: '오늘 비가 왔다.'")
print("  문장 B: '우산을 챙겨야겠다.'")
print("  예측: IsNext (두 문장이 논리적으로 연결됨)")

### 풀이 설명

- **핵심 개념**: MLM은 Cloze Test와 유사하며, NSP는 문장 관계 추론을 학습합니다.
- **대안**: RoBERTa는 NSP를 제거하고 더 많은 데이터로 MLM만 학습하여 성능을 개선했습니다.
- **실무 팁**: 문장 쌍을 다루는 태스크(문장 유사도, 자연어 추론)에서 NSP 학습이 도움됩니다.

---

## Q3. 토크나이저 사용하기

**문제**: 아래 텍스트를 토크나이저로 변환하고, 결과를 출력하세요.

```python
text = "Hugging Face 라이브러리는 정말 편리합니다!"
```

In [ ]:
# 정답
from transformers import AutoTokenizer

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

text = "Hugging Face 라이브러리는 정말 편리합니다!"

# 1. 토큰 리스트
tokens = tokenizer.tokenize(text)
print(f"1. 토큰 리스트: {tokens}")

# 2. 토큰 ID 리스트
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"2. 토큰 ID 리스트: {token_ids}")

# 3. attention_mask
encoded = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=20,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)
print(f"3. attention_mask: {encoded['attention_mask'][0].tolist()}")

In [ ]:
# 검증: 추가 분석
print("\n추가 분석:")
print(f"  총 토큰 수: {len(tokens)}")
print(f"  input_ids (특수 토큰 포함): {encoded['input_ids'][0].tolist()}")
print(f"  디코딩 결과: {tokenizer.decode(encoded['input_ids'][0], skip_special_tokens=True)}")

# WordPiece 토큰화 확인
print("\nWordPiece 토큰화 특징:")
for token in tokens:
    if token.startswith('##'):
        print(f"  '{token}': 이전 토큰의 연속 (subword)")

### 풀이 설명

- **접근 방법**: `tokenize()`로 토큰 분리, `convert_tokens_to_ids()`로 ID 변환, `encode_plus()`로 전체 인코딩
- **핵심 개념**: `##`로 시작하는 토큰은 WordPiece의 subword를 나타냅니다.
- **실수 주의**: `padding='max_length'` 사용 시 attention_mask에서 0은 패딩을 의미합니다.
- **실무 팁**: batch 처리 시 `tokenizer(texts, padding=True, truncation=True)`가 더 편리합니다.

---

## Q4. 모델 로드 및 출력 shape

**문제**: `bert-base-multilingual-cased` 모델을 로드하고, 아래 텍스트의 출력 shape을 확인하세요.

In [ ]:
# 정답
from transformers import AutoModel, AutoTokenizer

# 모델과 토크나이저 로드
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

text = "딥러닝 모델을 학습합니다."

# 토크나이징
inputs = tokenizer(text, return_tensors='pt')
print(f"입력 텍스트: {text}")
print(f"입력 토큰 수: {inputs['input_ids'].shape[1]}")

# 모델 순전파
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

# 출력 shape 확인
print(f"\n출력 shape:")
print(f"  last_hidden_state: {outputs.last_hidden_state.shape}")
print(f"    (batch_size={outputs.last_hidden_state.shape[0]}, "
      f"seq_len={outputs.last_hidden_state.shape[1]}, "
      f"hidden_size={outputs.last_hidden_state.shape[2]})")
print(f"  pooler_output: {outputs.pooler_output.shape}")
print(f"    ([CLS] 토큰의 출력, hidden_size={outputs.pooler_output.shape[1]})")

In [ ]:
# 검증: 토큰별 출력 확인
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
print("토큰별 hidden state:")
for i, token in enumerate(tokens):
    hidden = outputs.last_hidden_state[0, i, :5]  # 처음 5개 값만
    print(f"  {token:15s}: {hidden.tolist()}...")

### 풀이 설명

- **접근 방법**: `AutoModel.from_pretrained()`로 모델 로드, `model(**inputs)`로 순전파
- **핵심 개념**: 
  - `last_hidden_state`: 모든 토큰의 최종 hidden state (seq_len, hidden_size)
  - `pooler_output`: [CLS] 토큰의 hidden state에 추가 레이어 적용된 출력
- **대안**: `model.config.output_hidden_states=True`로 모든 레이어의 출력을 받을 수 있습니다.
- **실무 팁**: 분류 태스크에서는 `pooler_output`을, 토큰 분류에서는 `last_hidden_state`를 사용합니다.

---

## Q5. 특수 토큰 이해

**문제**: BERT의 특수 토큰 [CLS], [SEP], [PAD], [MASK]의 역할을 각각 설명하세요.

In [ ]:
# 정답
answer_q5 = """
BERT 특수 토큰의 역할:

1. [CLS] (Classification Token)
   - 위치: 모든 입력 시퀀스의 맨 앞
   - 역할: 전체 시퀀스의 집약된 표현을 담음
   - 용도: 문장 분류, 문장 유사도 등의 태스크에서 최종 출력으로 사용
   - 예시: [CLS] 오늘 날씨가 좋다 [SEP] -> [CLS]의 출력으로 긍/부정 분류

2. [SEP] (Separator Token)
   - 위치: 문장의 끝 또는 문장 사이
   - 역할: 문장의 경계를 구분
   - 용도: 단일 문장의 끝 표시, 문장 쌍의 구분
   - 예시: [CLS] 문장A [SEP] 문장B [SEP]

3. [PAD] (Padding Token)
   - 위치: 실제 토큰 이후의 빈 공간
   - 역할: 배치 내 시퀀스 길이를 동일하게 맞춤
   - 용도: attention_mask=0으로 모델이 무시하도록 함
   - 예시: [CLS] 안녕 [SEP] [PAD] [PAD] [PAD]

4. [MASK] (Mask Token)
   - 위치: MLM 사전학습 시 마스킹된 위치
   - 역할: 모델이 예측해야 할 토큰을 숨김
   - 용도: MLM 학습 (Fine-tuning 시에는 거의 사용 안 함)
   - 예시: 오늘 [MASK]가 좋다 -> '날씨' 예측
"""

print(answer_q5)

In [ ]:
# 검증: 특수 토큰 ID 확인
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

print("특수 토큰 ID:")
print(f"  [CLS]: {tokenizer.cls_token} (ID: {tokenizer.cls_token_id})")
print(f"  [SEP]: {tokenizer.sep_token} (ID: {tokenizer.sep_token_id})")
print(f"  [PAD]: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"  [MASK]: {tokenizer.mask_token} (ID: {tokenizer.mask_token_id})")
print(f"  [UNK]: {tokenizer.unk_token} (ID: {tokenizer.unk_token_id})")

### 풀이 설명

- **핵심 개념**: 특수 토큰은 모델이 입력 구조를 이해하는 데 필수적입니다.
- **실수 주의**: [PAD] 토큰에는 반드시 attention_mask=0을 설정해야 합니다.
- **실무 팁**: 문장 쌍 입력 시 `token_type_ids`로 어느 문장에 속하는지 구분합니다.

---

## Q6. 다운스트림 태스크 이해

**문제**: BERT를 사용한 텍스트 분류에서 왜 [CLS] 토큰의 출력을 사용하는지 설명하세요.

In [ ]:
# 정답
answer_q6 = """
[CLS] 토큰의 출력을 사용하는 이유:

1. 전체 시퀀스의 집약된 표현:
   - [CLS] 토큰은 Self-Attention을 통해 모든 토큰의 정보를 종합합니다.
   - 12개 레이어(BERT-base)를 거치면서 문장 전체의 의미가 [CLS]에 응축됩니다.
   - 따라서 [CLS] 출력 하나로 문장 전체를 대표할 수 있습니다.

2. 고정된 크기의 출력:
   - 문장 길이와 무관하게 항상 동일한 크기(hidden_size=768)의 벡터
   - 분류기(Linear Layer)에 바로 입력 가능

3. NSP 사전학습의 활용:
   - BERT는 NSP 과제에서 [CLS] 토큰으로 문장 관계를 예측하도록 학습됨
   - 이미 문장 수준의 표현을 담도록 사전학습되어 있음

4. 대안과 비교:
   - 모든 토큰 평균: 정보가 희석될 수 있음
   - 마지막 토큰: 위치에 따른 편향 발생
   - [CLS]: 위치가 고정되고 전체 정보를 담도록 학습됨
"""

print(answer_q6)

In [ ]:
# 검증: [CLS] 토큰 출력 확인
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("bert-base-multilingual-cased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

text = "이 영화는 정말 재미있었습니다."
inputs = tokenizer(text, return_tensors='pt')

model.eval()
with torch.no_grad():
    outputs = model(**inputs)

# [CLS] 토큰 출력
cls_output = outputs.last_hidden_state[:, 0, :]  # 첫 번째 토큰 = [CLS]
pooler_output = outputs.pooler_output  # pooler를 거친 출력

print(f"[CLS] hidden state shape: {cls_output.shape}")
print(f"pooler_output shape: {pooler_output.shape}")
print(f"\n차이: pooler_output은 [CLS]에 추가 Linear + Tanh 적용")

### 풀이 설명

- **핵심 개념**: [CLS]는 분류를 위해 설계된 특수 토큰으로, 전체 시퀀스 정보를 담습니다.
- **대안**: 문장 임베딩에서는 모든 토큰의 평균(mean pooling)이 더 좋을 수 있습니다.
- **실무 팁**: `AutoModelForSequenceClassification`은 내부적으로 pooler_output을 사용합니다.

---

## Q7. Fine-tuning 전략 비교

**문제**: "전체 Fine-tuning"과 "Feature Extraction" 전략의 차이점을 설명하고, 각각 언제 사용하면 좋은지 서술하세요.

In [ ]:
# 정답
answer_q7 = """
Fine-tuning 전략 비교:

1. 전체 Fine-tuning (Full Fine-tuning)
   - 방법: BERT 전체 레이어 + 분류 헤드 모두 학습
   - 장점:
     * 태스크에 최적화된 표현 학습 가능
     * 최고 성능 달성 가능
   - 단점:
     * 많은 연산량과 메모리 필요
     * 과적합 위험 (데이터가 적을 때)
     * 학습 시간이 오래 걸림
   - 사용 시점:
     * 레이블된 데이터가 충분할 때 (수천~수만 개 이상)
     * 도메인이 사전학습 데이터와 비슷할 때
     * 최고 성능이 필요할 때

2. Feature Extraction (특징 추출)
   - 방법: BERT 가중치 고정, 분류 헤드만 학습
   - 장점:
     * 빠른 학습 (수 분 내 완료)
     * 적은 메모리 사용
     * 과적합 방지
   - 단점:
     * 성능 상한이 존재
     * 태스크 특화 표현 학습 불가
   - 사용 시점:
     * 레이블된 데이터가 매우 적을 때 (수백 개 이하)
     * 빠른 프로토타이핑이 필요할 때
     * 컴퓨팅 자원이 제한적일 때

3. 선택 가이드:
   데이터 < 1,000개: Feature Extraction
   데이터 1,000~10,000개: 둘 다 시도 후 비교
   데이터 > 10,000개: 전체 Fine-tuning
"""

print(answer_q7)

In [ ]:
# 검증: 학습 가능 파라미터 수 비교
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=2
)

def count_params(model, trainable_only=False):
    if trainable_only:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    return sum(p.numel() for p in model.parameters())

print("파라미터 수 비교:")
print(f"  전체 파라미터: {count_params(model):,}")
print(f"  전체 Fine-tuning 시 학습 파라미터: {count_params(model, trainable_only=True):,}")

# BERT 고정
for param in model.bert.parameters():
    param.requires_grad = False

print(f"  Feature Extraction 시 학습 파라미터: {count_params(model, trainable_only=True):,}")
print(f"\n파라미터 비율: {count_params(model, trainable_only=True) / count_params(model) * 100:.4f}%")

### 풀이 설명

- **핵심 개념**: Fine-tuning은 데이터 양과 도메인 유사성에 따라 전략을 선택합니다.
- **대안**: Gradual Unfreezing은 상위 레이어부터 점진적으로 학습하는 중간 전략입니다.
- **실무 팁**: 먼저 Feature Extraction으로 빠르게 베이스라인을 만들고, 필요시 Fine-tuning으로 개선합니다.

---

## Q8. 한국어 토큰화

**문제**: 아래 두 문장을 토크나이저로 변환하고, 토큰화 결과를 비교 분석하세요.

In [ ]:
# 정답
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

text1 = "자연어 처리"
text2 = "Natural Language Processing"

tokens1 = tokenizer.tokenize(text1)
tokens2 = tokenizer.tokenize(text2)

print("토큰화 결과 비교:")
print("="*50)
print(f"\n한국어: '{text1}'")
print(f"  토큰: {tokens1}")
print(f"  토큰 수: {len(tokens1)}")

print(f"\n영어: '{text2}'")
print(f"  토큰: {tokens2}")
print(f"  토큰 수: {len(tokens2)}")

In [ ]:
# 분석
analysis = """
한국어와 영어 토큰화 차이점:

1. 토큰 수:
   - 한국어: 더 많은 토큰으로 분리되는 경향
   - 영어: 단어 단위로 비교적 적은 토큰
   - 이유: mBERT의 어휘(vocab)가 영어 중심으로 구성됨

2. 서브워드 분리:
   - 한국어: 한글 음절이 개별 토큰으로 분리되기 쉬움
   - 영어: 일반적인 단어는 통째로 유지됨

3. ##접두사:
   - 한국어에서 더 빈번하게 나타남
   - 어휘에 없는 한국어 단어가 조각남

4. 실무 시사점:
   - 한국어 특화 모델(KoBERT, KoELECTRA)이 더 효율적
   - mBERT는 범용성이 있지만 한국어 처리에 비효율적
   - max_length를 한국어에서는 더 길게 설정 권장
"""

print(analysis)

### 풀이 설명

- **핵심 개념**: mBERT의 WordPiece 어휘는 영어 중심이라 한국어가 더 많이 분리됩니다.
- **대안**: 한국어 특화 모델(klue/bert-base, monologg/kobert)을 사용하면 더 효율적입니다.
- **실무 팁**: 한국어 모델에서는 max_length를 영어보다 1.5~2배 길게 설정하는 것이 좋습니다.

---

## Q9. 감성 분류 모델 개선

**문제**: 본문의 NSMC 감성 분류 코드를 수정하여 다음을 구현하세요.

1. BERT 레이어를 고정(freeze)한 상태로 학습
2. 고정된 상태와 고정하지 않은 상태의 성능 비교

In [ ]:
# 정답
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

# 데이터 로드 (샘플)
try:
    train_df = pd.read_csv("../datasets/text/nsmc/ratings_train.txt", sep="\t")
    test_df = pd.read_csv("../datasets/text/nsmc/ratings_test.txt", sep="\t")
except:
    train_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt"
    test_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt"
    train_df = pd.read_csv(train_url, sep="\t")
    test_df = pd.read_csv(test_url, sep="\t")

train_df = train_df.dropna()
test_df = test_df.dropna()

# 샘플링
train_sample = train_df.sample(n=2000, random_state=42).reset_index(drop=True)
test_sample = test_df.sample(n=500, random_state=42).reset_index(drop=True)

print(f"훈련: {len(train_sample)}, 테스트: {len(test_sample)}")

In [ ]:
# 데이터셋 클래스
class NSMCDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        text = str(self.df.iloc[idx]['document'])
        label = self.df.iloc[idx]['label']
        
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# 학습 함수
def train_model(freeze_bert=False, epochs=2):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 모델 및 토크나이저
    tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
    model = AutoModelForSequenceClassification.from_pretrained(
        "bert-base-multilingual-cased",
        num_labels=2
    ).to(device)
    
    # BERT 고정 여부
    if freeze_bert:
        for param in model.bert.parameters():
            param.requires_grad = False
        print("BERT 레이어 고정됨 (Feature Extraction)")
    else:
        print("전체 Fine-tuning")
    
    # 데이터셋
    train_dataset = NSMCDataset(train_sample, tokenizer)
    test_dataset = NSMCDataset(test_sample, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16)
    
    # 옵티마이저
    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-5)
    
    # 학습
    train_losses = []
    eval_accs = []
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        train_losses.append(total_loss / len(train_loader))
        
        # 평가
        model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch in test_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
        eval_accs.append(correct / total)
        print(f"  Epoch {epoch+1}: Loss={train_losses[-1]:.4f}, Acc={eval_accs[-1]:.4f}")
    
    return train_losses, eval_accs

In [ ]:
# 비교 실험
print("=" * 50)
print("1. Feature Extraction (BERT 고정)")
print("=" * 50)
frozen_losses, frozen_accs = train_model(freeze_bert=True, epochs=2)

print("\n" + "=" * 50)
print("2. Full Fine-tuning (전체 학습)")
print("=" * 50)
full_losses, full_accs = train_model(freeze_bert=False, epochs=2)

In [ ]:
# 결과 비교 시각화
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=['Loss', 'Accuracy'])

# Loss
fig.add_trace(go.Scatter(y=frozen_losses, name='Feature Extraction', mode='lines+markers'), row=1, col=1)
fig.add_trace(go.Scatter(y=full_losses, name='Full Fine-tuning', mode='lines+markers'), row=1, col=1)

# Accuracy
fig.add_trace(go.Scatter(y=frozen_accs, name='Feature Extraction', mode='lines+markers', showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(y=full_accs, name='Full Fine-tuning', mode='lines+markers', showlegend=False), row=1, col=2)

fig.update_xaxes(title_text='Epoch')
fig.update_layout(title='Fine-tuning 전략 비교', height=400, template='plotly_white')
fig.show()

print(f"\n최종 정확도:")
print(f"  Feature Extraction: {frozen_accs[-1]:.4f}")
print(f"  Full Fine-tuning: {full_accs[-1]:.4f}")

### 풀이 설명

- **접근 방법**: `param.requires_grad = False`로 BERT 가중치를 고정합니다.
- **핵심 개념**: Feature Extraction은 빠르지만 성능 상한이 있고, Full Fine-tuning은 더 높은 성능을 달성합니다.
- **실수 주의**: 옵티마이저에 `filter(lambda p: p.requires_grad, model.parameters())`를 사용해야 합니다.
- **실무 팁**: 데이터가 적을 때는 Feature Extraction으로 시작하고, 데이터가 충분하면 Fine-tuning으로 개선합니다.

---

## Q10. 전체 파이프라인 구현

**문제**: Hugging Face의 `pipeline` API를 사용하여 감성 분석을 수행하는 코드를 작성하세요.

In [ ]:
# 정답
from transformers import pipeline
import torch

# 감성 분석 파이프라인 생성
# 한국어에 적합한 사전학습 모델 사용
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment",  # 다국어 감성 모델
    device=0 if torch.cuda.is_available() else -1
)

reviews = [
    "이 제품 정말 좋아요! 강추합니다.",
    "배송이 너무 늦고 포장이 엉망이었어요.",
    "가격 대비 괜찮은 것 같아요."
]

print("Hugging Face Pipeline 감성 분석 결과:")
print("="*60)
results = sentiment_pipeline(reviews)

for review, result in zip(reviews, results):
    # nlptown 모델은 1~5 별점으로 출력
    stars = int(result['label'].split()[0])
    sentiment = "긍정" if stars >= 4 else "부정" if stars <= 2 else "중립"
    print(f"\n리뷰: {review}")
    print(f"  별점: {'*' * stars} ({stars}/5)")
    print(f"  감성: {sentiment} (신뢰도: {result['score']:.2%})")

In [ ]:
# 대안: 학습된 모델로 파이프라인 생성
# (이전에 학습한 모델이 있다면)

# 방법 1: 모델 저장 후 로드
# model.save_pretrained("./my_sentiment_model")
# tokenizer.save_pretrained("./my_sentiment_model")
# my_pipeline = pipeline("sentiment-analysis", model="./my_sentiment_model")

# 방법 2: 모델과 토크나이저 직접 전달
# my_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

print("\nPipeline API 장점:")
print("  1. 간단한 인터페이스: 3줄로 추론 가능")
print("  2. 자동 전처리: 토크나이징, 패딩 자동 처리")
print("  3. 배치 처리: 여러 문장 한 번에 처리")
print("  4. GPU 지원: device 파라미터로 쉽게 설정")

### 풀이 설명

- **접근 방법**: `pipeline()`으로 간단하게 추론 파이프라인을 생성합니다.
- **핵심 개념**: Pipeline API는 전처리, 추론, 후처리를 자동으로 처리합니다.
- **대안**: 더 세밀한 제어가 필요하면 직접 토크나이저와 모델을 사용합니다.
- **실무 팁**: 프로덕션에서는 `pipeline`보다 직접 배치 처리가 더 효율적일 수 있습니다.

---

## 학습 정리

### 퀴즈 핵심 요약

| 퀴즈 | 핵심 개념 | 실무 적용 |
|------|----------|----------|
| Q1 | 양방향 문맥 이해 | 분류 vs 생성 모델 선택 |
| Q2 | MLM/NSP 사전학습 | 언어 이해 능력 학습 |
| Q3 | 토크나이저 사용 | 입력 데이터 준비 |
| Q4 | 모델 출력 shape | 다운스트림 태스크 연결 |
| Q5 | 특수 토큰 | 입력 형식 이해 |
| Q6 | [CLS] 토큰 | 문장 분류 구현 |
| Q7 | Fine-tuning 전략 | 데이터 양에 따른 선택 |
| Q8 | 다국어 토큰화 | 한국어 처리 최적화 |
| Q9 | 가중치 고정 | 효율적인 학습 |
| Q10 | Pipeline API | 빠른 프로토타이핑 |

### 실무 체크리스트

- [ ] 모델과 토크나이저는 같은 것 사용
- [ ] 데이터 양에 따라 Fine-tuning 전략 선택
- [ ] 학습률은 2e-5 ~ 5e-5로 설정
- [ ] Warmup과 Gradient Clipping 적용
- [ ] 한국어는 max_length 여유있게 설정
- [ ] 배포 시 모델 저장 및 Pipeline 활용